# EE 596 Mini Project 2
Blake Downey and Ryan Maroney

## Data Loading and Imports

In [ ]:
'''
Alternatively we can download directly from kaggle and don't need to mount the drive, but either should be fine. With Kaggle we would just need to update the key if I were to reset it for some reason, but this seemed to work for me
Source for instructions - https://www.kaggle.com/general/74235
Generated kaggle.json using echo rather than uploading it
'''
! pip install -q kaggle
! mkdir ~/.kaggle
! echo "{\"username\":\"ryanmaroney\",\"key\":\"b6917ffadd6b2af593b6802bb6721ace\"}" > ~/.kaggle/kaggle.json
! kaggle datasets list1
! kaggle competitions download -c twitter-sentiment-analysis-winter-2022
! unzip twitter-sentiment-analysis-winter-2022.zip
#! unzip twitter_sentiments_data.csv.zip
#! unzip twitter_sentiments_evaluation.csv.zip

usage: kaggle datasets [-h]
                       {list,files,download,create,version,init,metadata,status}
                       ...
kaggle datasets: error: argument command: invalid choice: 'list1' (choose from 'list', 'files', 'download', 'create', 'version', 'init', 'metadata', 'status')
 71% 57.0M/80.8M [00:00<00:00, 70.5MB/s]
100% 80.8M/80.8M [00:00<00:00, 101MB/s] 
Archive:  twitter-sentiment-analysis-winter-2022.zip
  inflating: twitter_sentiments_data.csv  
  inflating: twitter_sentiments_evaluation.csv  


In [ ]:
%%capture

import pandas as pd
import re, string
from nltk.tokenize import TweetTokenizer as TT
import nltk
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.feature_extraction.text import CountVectorizer as count_vec

from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

#!pip install tensorflow
import tensorflow
import tensorflow_hub as hub
!pip install tensorflow-text
import tensorflow_text as text
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall

# remove if we dont end up using stopwords
nltk.download('stopwords')
STOP_WORDS  = nltk.corpus.stopwords.words('english')

In [ ]:
# paths for train/val data and kaggle comp data
data_path = './twitter_sentiments_data.csv'
eval_path = './twitter_sentiments_evaluation.csv'

df_full = pd.read_csv(data_path)

In [ ]:
'''
Quick Data Analysis
'''
print(f'Shape of the data is: {df_full.shape}')
print(f'Column Names: {df_full.columns}')
print(f'The targets are: {df_full.target.unique()}')
print(f'Value Counts:\n{df_full["target"].value_counts()}')
print()


#Seems like the only important columns are target and tweet. 
df_full.head(20)

Shape of the data is: (1310106, 6)
Column Names: Index(['target', 'tweet_index', 'time stamp', 'Unnamed: 3', 'user', 'tweet'], dtype='object')
The targets are: [0 1]
Value Counts:
0    655081
1    655025
Name: target, dtype: int64



,target,tweet_index,time stamp,Unnamed: 3,user,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
3,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
4,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
5,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
6,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
7,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing
8,0,1467812579,Mon Apr 06 22:20:17 PDT 2009,NO_QUERY,pardonlauren,I just re-pierced my ears
9,0,1467812723,Mon Apr 06 22:20:19 PDT 2009,NO_QUERY,TLeC,@caregiving I couldn't bear to watch it. And ...


In [ ]:
df_important = df_full[['target','tweet']]
tweets_test = df_important['tweet']
df_important.head()

,target,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,"@nationwideclass no, it's not behaving at all...."
3,0,@Kwesidei not the whole crew
4,0,Need a hug


## Preprocessing

In [ ]:
'''
Functions to translate common twitter slang into actual english

Refernces on some of the common slang: 
https://saveti.kombib.rs/twerminology-twitter-slang-words-and-abbreviations#:~:text=egotwistical%3A%20Slang%20term%20used%20to%20describe%20a%20user,Shorthand%20versions%20of%20%22e-mail%22%20routinely%20used%20on%20Twitter.
https://slangit.com/terms/twitter

'''

tokenizer = TT(reduce_len=True)


def translate_slang(token):
  if token == 'u' or token == 'ya' : return 'you'
  elif token == 'n' : return 'and'
  elif token == 'ur' : return 'your'
  elif token == 'r' : return 'are'
  elif token == 'yrs' : return 'years'
  elif token == '2day' or token == '2-day' : return 'today'
  elif token == '2morow' or token == '2moro' : return 'tomorrow'
  elif token == '2nite' or token == '2-nite' : return 'tonight'
  elif token == 'some1' or token == 'sum1' : return 'someone'
  elif token == 'any1' : return 'anyone'
  elif token == 'pls' or token == 'plss' or token == 'plz' or token == 'plzz' or token == 'plzzz' : return 'please'
  elif token == 'ab' or token == 'abt' or token == 'bout' : return 'about'
  elif token == 'b4' or token == 'bfor' : return 'before'
  elif token == 'bc' : return 'because'
  elif token == 'cld' : return 'could'
  elif token == 'clk' : return 'click'
  elif token == 'chk' : return 'check'
  elif token == 'cre8' : return 'create'
  elif token == 'l8ter' : return 'later'
  elif token == 'da' : return 'the'
  elif token == 'deets' : return 'details'
  elif token == 'fab' : return 'fabulous'
  elif token == 'fav' : return 'favorite'
  elif token == 'oh' : return 'overheard'
  elif token == 'rt' : return 'retweet'
  elif token == 'sp' : return 'sponsored'
  elif token == 'wuz' or token == 'woz' : return 'was'
  elif token == 'wtv' : return 'whatever'
  elif token == 'sp' : return 'sponsored'
  elif token == 'w' : return 'with'
  elif token == 'oop' : return 'embarassed'
  elif token == 'sry' or token == 'srry' : return 'sorry'
  elif token == 'kk' or token == 'kewl' : return 'cool'
  elif token == 'puter' or token == "'puter" : return 'computer'
  elif token == 'hahaha' or token == 'hahahaha' : return 'haha'
  elif token == 'til' or token == 'till' or token == "'till" : return 'until'
  elif token == 'wut' or token == 'wat' or token == 'wah' : return 'what'
  elif token == 'fax': return 'truth'
  elif token == 'cap': return 'lie'
  elif token == 'tho': return 'though'
  elif token == 'thru': return 'through'
  elif token == 'cx': return 'correction'
  elif token == 'gn': return 'goodnight'
  elif token == 'tl': return 'timeline'
  elif token == 'nvm': return 'nevermind'
  elif token == 'aaw' or token == 'aww': return 'aw'
  elif token == 'fuk' or token == 'fuq': return 'fuck'
  elif token == 'helloo' or token == 'hellooo' : return 'hello'

  return token

def special_case(tweet):
  tweet = tweet.split()
  sent = []
  for w in tweet:
    if w == 'b/c':
      sent.append('because')
    elif w == 'w/o' or w == 'w/out':
      sent.append('with')
      sent.append('out')
    elif  w == 'w/u' or w == 'w/you':
      sent.append('with')
      sent.append('you')
    else: sent.append(w)
  return ' '.join(sent)


def translate(tweet):
  out = [] #list of words

  tweet = re.sub("(@[A-Za-z0-9_]+)","", tweet) # remove mentions
  tweet = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', tweet)
  #handle special cases before tokenizer ie words like 'b/c' that have a '/' in it
  tweet = special_case(tweet)
  words = tokenizer.tokenize(tweet) # use the tokenizer to split the words/punctuations 
  
  # translate abbreviations into multi words
  for word in words:
    if word not in string.punctuation:# and word not in STOP_WORDS:
      word = word.lower()
      if word == 'idk':
        out.append('i')
        out.append('dont')
        out.append('know')
      elif word == 'wcw':
        out.append('women')
        out.append('crush')
        out.append('wednesday')
      elif word == 'mcm':
        out.append('man')
        out.append('crush')
        out.append('monday')
      elif word == 'gonna' or word == 'gona':
        out.append('going')
        out.append('to')
      elif word == 'ftb':
        out.append('fuck')
        out.append('that')
        out.append('bitch')
      elif word == 'fym':
        out.append('fuck')
        out.append('you')
        out.append('mean')
      elif word == 'gtfo':
        out.append('get')
        out.append('the')
        out.append('fuck')
        out.append('out')
      elif word == 'nbd':
        out.append('no')
        out.append('big')
        out.append('deal')
      elif word == 'stfu':
        out.append('shut')
        out.append('the')
        out.append('fuck')
        out.append('up')
      elif word == 'fml':
        out.append('fuck')
        out.append('my')
        out.append('life')
      elif word == 'lol':
        out.append('laugh')
        out.append('out')
        out.append('loud')
      elif word == 'rofl':
        out.append('rolling')
        out.append('on')
        out.append('the')
        out.append('floor')
        out.append('laughing')
      elif word == 'nsfw':
        out.append('not')
        out.append('safe')
        out.append('for')
        out.append('work')
      elif word == 'sfw':
        out.append('safe')
        out.append('for')
        out.append('work')
      elif word == 'fwb':
        out.append('friends')
        out.append('with')
        out.append('benefits')
      elif word == 'fyi':
        out.append('for')
        out.append('your')
        out.append('information')
      elif word == 'nm':
        out.append('not')
        out.append('much')
      elif word == 'brb':
        out.append('be')
        out.append('right')
        out.append('back')
      elif word == 'ikr':
        out.append('i')
        out.append('know')
        out.append('right')
      elif word == 'pos':
        out.append('piece')
        out.append('of')
        out.append('shit')
      elif word == 'ootd':
        out.append('outfit')
        out.append('of')
        out.append('the')
        out.append('day')
      elif word == 'lmao':
        out.append('laugh')
        out.append('my')
        out.append('ass')
        out.append('off')
      elif word == 'hags':
        out.append('have')
        out.append('a')
        out.append('great')
        out.append('summer')
      elif word == 'jk':
        out.append('just')
        out.append('kidding')
      elif word == 'tbt':
        out.append('throwback')
        out.append('thursday')
      elif word == 'rn':
        out.append('right')
        out.append('now')
      elif word == 'tbh':
        out.append('to')
        out.append('be')
      elif word == 'gl':
        out.append('good')
        out.append('luck')
        out.append('honest')
      elif word == 'tftf':
        out.append('thanks')
        out.append('for')
        out.append('the')
        out.append('follow')
      elif word == 'tfti':
        out.append('thanks')
        out.append('for')
        out.append('the')
        out.append('invite')
      elif word == 'smh':
        out.append('shake')
        out.append('my')
        out.append('head')
      elif word == 'smh':
        out.append('shake')
        out.append('my')
        out.append('head')
      elif word == 'idc':
        out.append('i')
        out.append('dont')
        out.append('care')
      elif word == 'mfs':
        out.append('mother')
        out.append('fuckers')
      elif word == 'fr':
        out.append('for')
        out.append('real')
      elif word == 'wtf':
        out.append('what')
        out.append('the')
        out.append('fuck')
      elif word == 'mtf':
        out.append('more')
        out.append('to')
        out.append('follow')
      elif word == 'nts':
        out.append('note')
        out.append('to')
        out.append('self')
      elif word == 'ic':
        out.append('i')
        out.append('see')
      elif word == 'ftl':
        out.append('for')
        out.append('the')
        out.append('loss')
      elif word == 'ftw':
        out.append('for')
        out.append('the')
        out.append('win')
      elif word == 'fomo':
        out.append('fear')
        out.append('of')
        out.append('missing')
        out.append('out')
      elif word == 'f2f' or word == 'ftf':
        out.append('face')
        out.append('to')
        out.append('face')
      elif word == 'irl':
        out.append('in')
        out.append('real')
        out.append('life')
      elif word == 'tmi':
        out.append('to')
        out.append('much')
        out.append('information')
      elif word == 'omg':
        out.append('ohh')
        out.append('my')
        out.append('gosh')
      elif word == 'psa':
        out.append('public')
        out.append('service')
        out.append('announcement')
      elif word == 'abg':
        out.append('asian')
        out.append('baby')
        out.append('girl')
      elif word == 'abb':
        out.append('asian')
        out.append('baby')
        out.append('boy')
      elif word == 'tfw':
        out.append('that')
        out.append('feeling')
        out.append('when')
      elif word == 'twart':
        out.append('twitter')
        out.append('art')
      elif word == 'yolo':
        out.append('you')
        out.append('only')
        out.append('live')
        out.append('once')
      elif word == 'yoyo':
        out.append("you")
        out.append("are")
        out.append('on')
        out.append('your')
        out.append('own')
      elif word == 'hifw':
        out.append('how')
        out.append('i')
        out.append('felt')
        out.append('when')
      elif word == 'ngl':
        out.append('not')
        out.append('going')
        out.append('to')
        out.append('lie')
      elif word == 'icymi':
        out.append('in')
        out.append('case')
        out.append('you')
        out.append('missed')
        out.append('it')
      elif word == 'wouldnt' or word == "wouldn't":
        out.append('would')
        out.append('not')
      elif word == 'cant' or word == "can't":
        out.append('can')
        out.append('not')
      elif word == 'didnt' or word == "didn't":
        out.append('did')
        out.append('not')
      elif word == 'couldnt' or word == "couldn't":
        out.append('could')
        out.append('not')
      elif word == 'wo':
        out.append('with')
        out.append('out')
      elif word == 'imy' or word == 'imu':
        out.append('i')
        out.append('miss')
        out.append('you')
      elif word == 'ily' or word == 'ilyy':
        out.append('i')
        out.append('love')
        out.append('you')
      elif word == 'wth':
        out.append('what')
        out.append('the')
        out.append('hell')
      elif word == 'pc':
        out.append('politically')
        out.append('correct')
      elif word == 'imo':
        out.append('in')
        out.append('my')
        out.append('opinion')
      elif word == 'sob':
        out.append('son')
        out.append('of')
        out.append('a')
        out.append('bitch')
      elif word == 'wanna' or word == 'wana':
        out.append('want')
        out.append('to')
      elif word == 'rip':
        out.append('rest')
        out.append('in')
        out.append('peace')
      elif word == 'dm' or word == 'dms' or word == "dm's":
        out.append('direct')
        out.append('message')
      elif word == 'f4f':
        out.append('follow')
        out.append('for')
        out.append('follow')
      elif word == 'atm':
        out.append('at')
        out.append('the')
        out.append('moment')
      elif word == "i'm":
        out.append('i')
        out.append('am')
      elif word == "it's" or word == 'its':
        out.append('it')
        out.append('is')
      elif word == "i've":
        out.append('i')
        out.append('have')
      elif word == 'el-stupido':
        out.append('the')
        out.append('stupid')
      elif word == '..' or word == '...' :
        continue
      else: 
        out.append(word)

  # call translate slang to convert abbreviated words to single words
  for i,word in enumerate(out):
    out[i] = translate_slang(word)

  return ' '.join(out) #return out if you want tokenized output - otherwise return the joined list of words

In [ ]:
# testing the above function
test_sentence = 'hellooooo hahaha wtf i cant go w/o u to da party, sry bout it'
print('Testing on sample sentence...')
print(f'test sentence: {test_sentence}')
print(f'translated: {translate(test_sentence)}\n\n')

print('Testing on actual twitter data...')
for tweet in tweets_test[100:110]:
  print(f'original: {tweet}')
  print(f'translated: {translate(tweet)}\n')

Testing on sample sentence...
test sentence: hellooooo hahaha wtf i cant go w/o u to da party, sry bout it
translated: hello haha what the fuck i can not go with out you to the party sorry about it


Testing on actual twitter data...
original: ugh. cant sleep. its 1:30am. 
translated: ugh can not sleep it is 1:30 am

original: Hanging in Crooners. Wanna sing. Can't. Sucks. 
translated: hanging in crooners want to sing can not sucks

original: @eRRe_sC aaw i miss ya all too.. im leaving to BH tomorrow &quot;morning&quot; i think.. aww i wanna go to the beach w u girls!! 
translated: aw i miss you all too im leaving to bh tomorrow morning i think aw i want to go to the beach with you girls

original: Is pissed off that there's no ASBA's for a radio station. 
translated: is pissed off that there's no asba's for a radio station

original: wednesday my b-day n don't know what 2 do!  
translated: wednesday my b-day and don't know what 2 do

original: I know my life has been flipped upside do

In [ ]:
# dividing factor
sub_sample_ratio = 1

# separate to subsample indiviudally
df_1 = df_important[df_important['target'] == 1]
df_0 = df_important[df_important['target'] == 0]

# length of target classes 
pos_count = len(df_1)
neg_count = len(df_0)

# subsample the length
df_1 = df_1.iloc[:int(pos_count/sub_sample_ratio)]
df_0 = df_0.iloc[:int(neg_count/sub_sample_ratio)]

# concat so the df has both classes again
df = pd.concat([df_0, df_1])

# apply the translate function to the tweets
df['tweet'] = df['tweet'].apply(translate)

In [ ]:
SEED = 53

# split the data into train and val
X_train, X_val, y_train, y_val = tts(df['tweet'], df['target'], test_size=0.2, random_state=SEED)

# initiate the vectorizer
#vec = tfidf(max_features=100000, ngram_range=[1,4])
vec = count_vec(max_features=100000, ngram_range=[1,4])

# transform the train/validation data with the tfidf vectorizer
X_train = vec.fit_transform(X_train)
X_val = vec.transform(X_val)


## Basline Models

In [ ]:
'''
BASELINE -- non deep learning

Test LR/SGD Classifier with a TfidfVectorizer
'''
print(f'Shape of Train Data: {X_train.shape}')
print(f'Shape of Validation Data: {X_val.shape}')

# fit the models
model_LR = LogisticRegression(penalty='l2', max_iter=500, C=0.1, random_state=SEED).fit(X_train, y_train)
model_SGD = SGDClassifier(penalty='l2').fit(X_train, y_train)

# preds for logistic regression
train_pred_LR = model_LR.predict(X_train)
val_pred_LR = model_LR.predict(X_val)

# preds for sgd classifier
train_pred_SGD = model_SGD.predict(X_train)
val_pred_SGD = model_SGD.predict(X_val)

# print results
print('\nLR Results:')
print(f'Train Accuracy: {accuracy_score(y_train, train_pred_LR)*100 : .2f}%')
print(f'Validation Accuracy: {accuracy_score(y_val, val_pred_LR)*100 : .2f}%')
print(f'Train f1: {f1_score(y_train, train_pred_LR): .5f}')
print(f'Validation f1: {f1_score(y_val, val_pred_LR): .5f}')

print('\nSGD Results:')
print(f'Train Accuracy: {accuracy_score(y_train, train_pred_SGD)*100 : .2f}%')
print(f'Validation Accuracy: {accuracy_score(y_val, val_pred_SGD)*100 : .2f}%')
print(f'Train f1: {f1_score(y_train, train_pred_SGD): .5f}')
print(f'Validation f1: {f1_score(y_val, val_pred_SGD): .5f}')

Shape of Train Data: (1048084, 100000)
Shape of Validation Data: (262022, 100000)

LR Results:
Train Accuracy:  84.12%
Validation Accuracy:  81.99%
Train f1:  0.84398
Validation f1:  0.82366

SGD Results:
Train Accuracy:  82.32%
Validation Accuracy:  81.16%
Train f1:  0.82628
Validation f1:  0.81533


In [ ]:
'''
Baseline Results with the evaluation set for kaggle
'''

df_eval = pd.read_csv(eval_path)

df_eval = df_eval[['tweet_index','tweet']]

# apply the translate function
df_eval['tweet'] = df_eval['tweet'].apply(translate)

# apply vectorizer
vectorized_eval = vec.transform(df_eval['tweet'])

# predict
pred_eval = model_LR.predict(vectorized_eval)

out_data = {
    'tweet_index' : (df_eval['tweet_index']),
    'target' : (pred_eval)
}

out_df = pd.DataFrame(out_data, index=None)

out_df.to_csv('preds.csv')

## BERT Based Model

In [ ]:
X_train, X_val, y_train, y_val = tts(df['tweet'], df['target'], test_size=0.2, random_state=SEED)

In [ ]:
'''
BERT HERE

Reference - https://www.analyticsvidhya.com/blog/2021/12/text-classification-using-bert-and-tensorflow/
'''
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
# Bert layers
text_input = tensorflow.keras.layers.Input(shape=(), dtype=tensorflow.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tensorflow.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tensorflow.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tensorflow.keras.Model(inputs=[text_input], outputs = [l])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import os.path

# Download pre-trained model if available
if os.path.exists("test_bert_model_epoch_1_chunk_94.h5"):
  model = tensorflow.keras.models.load_model("test_bert_model_epoch_1_chunk_94.h5", custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [ ]:
# BERT has roughly 110 Million parameters, training on only a subset in final notebook for reasonable training time and to demonstrate training
subset_percent = 0.05
subset_number = int(len(X_train) * subset_percent)
model.fit(X_train.iloc[:subset_number], y_train.iloc[:subset_number], validation_data=(X_val,y_val), epochs=1, batch_size = 32)

1638/1638 [==============================] - 6156s 4s/step - loss: 0.5391 - accuracy: 0.7298 - val_loss: 0.5093 - val_accuracy: 0.7560


In [ ]:

df_eval = pd.read_csv(eval_path)

df_eval = df_eval[['tweet_index','tweet']]

# apply the translate function
df_eval['tweet'] = df_eval['tweet'].apply(translate)
tweets = df_eval['tweet']

predictions = model.predict(tweets)

predictions = np.round(np.array(predictions))

out_data = {
    'tweet_index' : (df_eval['tweet_index']),
    'target' : (predictions.flatten())
}

out_df = pd.DataFrame(out_data, index=None)

out_df.to_csv('bert_predictions.csv')

## Bi-LSTM Based Model

In [ ]:
tokener = Tokenizer(num_words=20000, split=' ') # max 10000 words and split the string on space
def token_and_pad(tweets):
    tokener.fit_on_texts(tweets)
    data = tokener.texts_to_sequences(tweets)
    data = pad_sequences(data, padding='post', maxlen=50)
    return data

In [ ]:
# convert to tokens and pad each sequence to the same length
data = token_and_pad(df['tweet']) 

# this makes it go beep boop bop
target = pd.get_dummies(df['target']) 

In [ ]:
# train test split this bitttttch
SEED = 42
X_train, X_val, y_train, y_val = tts(data, target, test_size=0.1, random_state=SEED)

print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_val: {X_val.shape}')

Shape of X_train: (1179095, 50)
Shape of X_val: (131011, 50)


In [ ]:
'''
Bi-LSTM
'''
def build_model():
    # bi - lstm deep learning architecture
    model = Sequential()
    model.add(layers.Embedding(20000, 32, input_length=50))
    model.add(layers.Conv1D(500, 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling1D())
    model.add(layers.Bidirectional(layers.LSTM(128)))
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dropout(0.58))
    model.add(layers.Dense(2, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) #, Precision(), Recall()
    
    return model

In [ ]:
from keras.callbacks import ModelCheckpoint

model_name = './models/best_rmsprop3.epoch{epoch:02d}-val_loss{val_loss:.2f}.h5'

chkpoint = ModelCheckpoint(filepath=model_name, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

callbacks = [chkpoint]

In [ ]:
model = build_model()

In [ ]:
epochs = 10
hist = model.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=1024, epochs=epochs, callbacks=callbacks)

Epoch 1/10
1152/1152 [==============================] - ETA: 0s - loss: 0.4352 - accuracy: 0.7989
Epoch 00001: val_loss improved from inf to 0.41024, saving model to ./models\best_rmsprop3.epoch01-val_loss0.41.h5
1152/1152 [==============================] - 639s 552ms/step - loss: 0.4352 - accuracy: 0.7989 - val_loss: 0.4102 - val_accuracy: 0.8125
Epoch 2/10
1152/1152 [==============================] - ETA: 0s - loss: 0.3876 - accuracy: 0.8256
Epoch 00002: val_loss improved from 0.41024 to 0.38200, saving model to ./models\best_rmsprop3.epoch02-val_loss0.38.h5
1152/1152 [==============================] - 668s 580ms/step - loss: 0.3876 - accuracy: 0.8256 - val_loss: 0.3820 - val_accuracy: 0.8274
Epoch 3/10
1152/1152 [==============================] - ETA: 0s - loss: 0.3698 - accuracy: 0.8352
Epoch 00003: val_loss improved from 0.38200 to 0.37439, saving model to ./models\best_rmsprop3.epoch03-val_loss0.37.h5
1152/1152 [==============================] - 663s 575ms/step - loss: 0.3698 - a

In [ ]:
print('Train vs. Validation loss')
x = np.arange(0, len(hist.history['loss']))
plt.plot(x,hist.history['loss'], color='red', label='train')
plt.plot(x,hist.history['val_loss'], label='validation')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc='upper right')
plt.show()

print('\nTrain vs. Validation Accuracy')
x = np.arange(0, len(hist.history['multi_acc']))
plt.plot(x,hist.history['multi_acc'], color='red', label='train')
plt.plot(x,hist.history['val_multi_acc'], label='validation')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
model = build_model()
best_model = './models/best_rmsprop3.epoch05-val_loss0.37.h5'
model.load_weights(best_model)

In [ ]:
'''
lstm Results with the evaluation set for kaggle
'''
df_eval = pd.read_csv(eval_path)

df_eval = df_eval[['tweet_index','tweet']]

# apply the translate function
df_eval['tweet'] = df_eval['tweet'].apply(translate)
tweets = df_eval['tweet']

# apply tokener
data = tokener.texts_to_sequences(tweets)
data = pad_sequences(data, padding='post', maxlen=50)

In [ ]:
# predict
pred_eval =model.predict(data)

preds = []
for pred in pred_eval:
    preds.append(np.argmax(pred))
    
out_data = {
    'tweet_index' : (df_eval['tweet_index']),
    'target' : (preds)
}

out_df = pd.DataFrame(out_data, index=None)

out_df.to_csv('./preds.csv')